In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import json
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

with open("/content/drive/My Drive/Colab Notebooks/linear_.txt",'r') as file_words:
  vocab = sorted([i.replace("•","").strip().lower() for i in file_words.read().split("\n") if i], key=lambda x: -len(x))
  vocab = [[i[:i.rindex(" ")], i[i.rindex(" ")+1:]] for i in vocab]
with open("/content/drive/My Drive/Colab Notebooks/Strang-Linear Algebra.txt",'r') as file_text:
  book = file_text.read().lower()

with open("/content/drive/My Drive/Colab Notebooks/linear_.txt",'w') as file_words:
  file_words.write("\n".join(sorted([" ".join(i) for i in vocab])))

In [29]:
vocab[:20]

[['closure under addition and scalar multiplication', 'act'],
 ['negative semidefinite quadratic form', 'typ'],
 ['positive semidefinite quadratic form', 'typ'],
 ['general solution to a linear system', 'obj'],
 ['determinant test for invertibility', 'act'],
 ['orthogonally diagonalizable matrix', 'obj'],
 ['infinite-dimensional vector space', 'obj'],
 ['negative definite quadratic form', 'typ'],
 ['positive definite quadratic form', 'typ'],
 ['complex euclidean inner product', 'obj'],
 ['finite-dimensional vector space', 'obj'],
 ['eigendecomposition of a matrix', 'act'],
 ['gram-schmidt orthogonalization', 'act'],
 ['intersection of vector spaces', 'obj'],
 ['orthogonal change of variable', 'act'],
 ['augmented coefficient matrix', 'obj'],
 ['best rank one approximations', 'act'],
 ['singular value decomposition', 'trm'],
 ['least squares approximation', 'act'],
 ['approximation of functions', 'trm']]

In [3]:
count = 0

TRAIN_DATA = [
    ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
    ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
]

data = []
sentences = []

for sentence in book.split("."):  
  sentence = sentence.replace("\n"," ").replace("¤","").replace("§","").strip()
  if len(sentence) > 25 and list(sentence).count(" ") < len(sentence)/3:
    words = []
    sentence = " ".join(sentence.split())
    taken = []
    for word in vocab:
      index = sentence.find(word[0])
      if index != -1 and index not in taken and (len(sentence) <= index+len(word[0])+2 or not sentence[index+len(word[0])].isalpha() or sentence[index+len(word[0])] == "s") :
        words.append([index, index+len(word[0]),word[1]])
        taken.extend([i for i in range(index, index+len(word[0]))])
      # elif index != -1:
        # print(word, "NOT THERE")
    if words:
      data.append((sentence, {"entities": words}))
    sentences.append(sentence)
print(data[:10])

['line', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['matrix', 'obj'] NOT THERE
['row', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['subspace', 'obj'] NOT THERE
['orthogonal', 'typ'] NOT THERE
['orthogonal', 'typ'] NOT THERE
['square', 'typ'] NOT THERE
['diagonal', 'typ'] NOT THERE
['difference', 'obj'] NOT THERE
['matrix', 'obj'] NOT THERE
['norm', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['factor', 'obj'] NOT THERE
['matrix', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['row', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['norm', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['dimension', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['dimension', 'obj'] NOT THERE
['line', 'obj'] NOT THERE
['square', 'typ'] NOT THERE
['line', 'obj'] NOT THERE
['row', 'obj'] NOT THERE
['matrix', 'obj'] NOT THERE
['rank', 'obj'] NOT THERE
['line', 'obj'] N

In [0]:
# finding other terms not in vocabulary among words in the books

# nltk.download('stopwords')
# from nltk.corpus import stopwords

# sent = " ".join(sentences)
# for i in vocab:
#   sent = sent.replace(i[0],"")

# stop_words = set(stopwords.words('english')) 
# pos_data = nltk.ConditionalFreqDist((word.lower(), tag) for (word, tag) in nltk.pos_tag(nltk.word_tokenize(sent)))
# for word in pos_data.conditions():
#   if len(word) > 3 and word.isalpha() and word not in stop_words:
#     tags = pos_data[word].keys()
#     print(word, ' '.join(tags))

In [0]:
from __future__ import unicode_literals, print_function
import pickle
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

LABEL = ["act","obj",'trm','typ']

In [7]:
TRAIN_DATA = data

model=None
new_model_name='new_model'
output_dir="/content/drive/My Drive/Colab Notebooks/"
n_iter=10

if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")

# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")

# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):  # only train NER
    # reset and initialize the weights randomly – but only if we're
    # training a new model
    if model is None:
        nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                losses=losses,
            )
        print("Losses", losses)

# test the trained model
for text, _ in TRAIN_DATA[:50]:
    doc = nlp(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    # print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

# save model to output directory
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

    # test the saved model
    print("Loading from", output_dir)
    nlp2 = spacy.load(output_dir)
    # for text, _ in TRAIN_DATA:
    #     doc = nlp2(text)
    #     print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    #     print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

# main()


Created blank 'en' model
Losses {'ner': 7808.924238063134}
Losses {'ner': 4669.9797024565205}
Losses {'ner': 3890.9170818624243}
Losses {'ner': 3559.5186795285085}
Losses {'ner': 3264.1628489188593}
Losses {'ner': 3107.716359756003}
Losses {'ner': 2875.7062737949404}
Losses {'ner': 2676.8989400528762}
Losses {'ner': 2622.0714341987705}
Losses {'ner': 2493.4614859764815}
Entities [('invertible matrix', 'obj')]
Tokens [('(', '', 2), ('b', '', 2), (')', '', 2), ('every', '', 2), ('invertible', 'obj', 3), ('matrix', 'obj', 1), ('can', '', 2), ('be', '', 2), ('diagonalized', '', 2)]
Entities [('line', 'obj')]
Tokens [('in', '', 2), ('this', '', 2), ('example', '', 2), ('the', '', 2), ('mirror', '', 2), ('is', '', 2), ('the', '', 2), ('45', '', 2), ('°', '', 2), ('line', 'obj', 3), ('y', '', 2), ('=', '', 2), ('x', '', 2), (',', '', 2), ('and', '', 2), ('a', '', 2), ('point', '', 2), ('like', '', 2), ('(', '', 2), ('2;2', '', 2), (')', '', 2), ('is', '', 2), ('unchanged', '', 2)]
Entities [(

In [35]:
with open("/content/drive/My Drive/Colab Notebooks/test.txt") as filee:
    test_text = filee.read()
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)


Entities in '(One may verify this also by carrying out the matrix multiplications.) Using (1)
and (2) we obtain
det(Pi j A) = det 
Gi j(1)(Gi j(−1))T Gi j(1)Mj(−1)A

= det(Gi j(1)) det((Gi j(−1))T ) det(Gi j(1)) det(Mj(−1)) det(A)
= (−1) det(A). 	
Since det(A) = det(AT ) (cp. (5) in Lemma 7.10), the results in Lemma 7.13 for
the rows of A can be formulated analogously for the columns of A.
Example 7.14 Consider the matrices
A simple calculation shows that det(A) = −4. Since B is obtained from A by
exchanging the first two columns we have det(B) = − det(A) = 4.
The determinant map can be interpreted as a map of (Rn,1)n to R, i.e., as a map of
the n columns of the matrix A ∈ Rn,n to the ring R. If ai, aj ∈ Rn,1 are two columns
of A,
by (3) in Lemma 7.13. Due to this property the determinant map is called an alternating map of the columns of A. Analogously, the determinant map is an alternating
map of the rows of A.
This property is called the linearity of the determinant map with respe

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer 
def distractors(chapter_sentences, key, sentence):
  candidates = []
  cand_sents = []
  nlp2 = spacy.load(output_dir)
  key_nlp = nlp2(key)

  label = key_nlp.ents

  tfidf_vectorizer=TfidfVectorizer()
  tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(chapter_sentences)
  tag = tfidf_vectorizer.get_feature_names()
  n = tfidf_vectorizer_vectors.sum(axis=0).A1
  candidates_scores = []
  result = dict(zip(tag,n))

  key_score = sum([result[key.split()[i]] for i in range(len(key.split()))])/len(key.split())
  print(key_score)
  max_score = [- float("inf"),""]
  key_tags = []

  sent = sentence
  context = sent[:sent.index(key)] + "." +sent[sent.index(key)+len(key):]
  print(context)
  context = context.split(".")[0].split()[-2:] + context.split(".")[1].split()[:2]
  print(context)
  for word in context:
      tag = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
      key_tags.append(tag)

  if label:
    key_label = label[0].label_
  for sentence in chapter_sentences:
    doc = nlp2(sentence)
    for ent in doc.ents:
      if ent.label_ == key_label:
          candidates.append(ent.text)
          cand_sents.append(sentence)
          print(ent.label_, ent.text)

  for cand, sent in zip(candidates, cand_sents):
    if cand not in key and key not in cand:
      sent_simil = sum([2 for i in sent if i in sentence])/(len(sentence.split())+len(sent.split())) #sentence_similarity()
      cand_tags = 0
      context = sent[:sent.index(cand)] + "." +sent[sent.index(cand)+len(cand):]
      print(context)
      context = context.split(".")[0].split()[-2:] + context.split(".")[1].split()[:2]
      print(context)
      for word in [0,1,2,3]:
        tag = nltk.pos_tag(nltk.word_tokenize(context[word]))[0][1]
        cand_tags -= int(tag != key_tags[word])                                                     #context_similarity()

      diff_score = (sum([result[cand.split()[i]] for i in range(len(cand.split()))])/len(cand.split()) - key_score + 1)/2   #importance_difference()
      score = [sent_simil + cand_tags/4 - diff_score, cand]
      max_score = max([max_score, score],key=lambda x: x[0])
  print(max_score)
    
  



In [113]:
distractors(['this is a symmetric matrix and also a plane is here','this zero matrix can be invertible','use matrix multiplication for this problem'], 'symmetric matrix', 'this is a symmetric matrix and also a plane is here')

0.5278877930684712
this is a . and also a plane is here
['is', 'a', 'and', 'also']
obj symmetric matrix
obj plane
obj matrix
this is a symmetric matrix and also a . is here
['also', 'a', 'is', 'here']
[4.868090043234419, 'plane']
